In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [2]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [3]:
cell_demand = 0.001*164.98*(1.369*1e6)*2           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

# D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [114]:
# penalty = 10000 # ton/yr
penalty = 6; # ton/yr

In [5]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [6]:
ipt = []
ipt = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount = ipt

15-element Vector{Float64}:
  49670.291589732
  69512.470736544
  67696.846500888
  69382.78329114
 129687.44540400001
  72376.074329
   5239.896784
   1309.974196
  31111.887154999997
  60258.81301599999
 327493.549
   2258.5762
   1806.86096
 451715.24
 451715.24

In [108]:
D = [0.5, 0.7, 0.8, 1, 1.1, 1.2, 1.4]*cell_demand

7-element Vector{Float64}:
 225857.62
 316200.66799999995
 361372.19200000004
 451715.24
 496886.764
 542058.288
 632401.3359999999

### Solve subproblem to get dual $\pi$ and $\alpha$

In [115]:
function reformu_pi(col_idx, proc_idx, ls_cstr)
    pi_matrix = zeros(ncty, nproc)
    pi = [getdual(con) for con in ls_cstr]
    B = []
    for k in proc_idx
        append!(B, scaler[k] * ones(ncty))
    end
    pi = pi .* B
    pi = reshape(pi, ncty, size(proc_idx)[1])
    pi_matrix[:,col_idx] = sum(pi, dims=2)
    
    return pi_matrix
end


# cathode_pi = zeros(ncty, nproc)
# cth_pi = [getdual(con) for con in cstr_cth]
# cth_B = []
# for k in cathode
#     push!(cth_B, scaler[k] * ones(ncty))
# end
# cth_pi = cth_pi .* cth_B
# cth_pi = reshape(cth_pi, ncty, size(cathode)[1])
# cathode_pi[:,5] = sum(cth_pi, 2)

reformu_pi (generic function with 1 method)

In [116]:
function subprob(x_hat, d)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, s >= 0)

    # node output flow constraint
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x_hat[i,k]) for k in 1:nproc for i in 1:ncty]

    # cathode LCA constraints (index=5)
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]

    # cell LCA constraints (index=11)
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]

    # non cell LCA constraints (index=14)
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]

    # battery LCA constraints (index=15)
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    
    @constraint(model, cstr_alp, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + s >= d)
    for j in 1:ncty-1
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0)
    end 
    
    
    #######################
    tranD = Vector{AffExpr}(undef, ncty)
    tranS= Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
        end
        tranD[j] = arc_emi
        tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
    end
    
    
    @objective(model, Min, sum(tranD-tranS) + s*penalty)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_s = JuMP.value.(s)
    
    
    ####################### 
    op_pi = [getdual(con) for con in cstr_op]
    op_pi = reshape(op_pi, ncty, nproc)

    cth_pi = reformu_pi(5, cathode, cstr_cth)
    cell_pi = reformu_pi(11, cell, cstr_cell)
    noncell_pi = reformu_pi(14, noncell, cstr_noncell)
    battery_pi = reformu_pi(15, battery, cstr_battery)
    pi_matrix = op_pi + cth_pi + cell_pi + noncell_pi + battery_pi
    
    alp = getdual(cstr_alp)

    
    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "s_opt"=>sub_s, "pi"=>pi_matrix, "alp"=>alp]);
    return result
    
end


subprob (generic function with 1 method)

### Solve master problem

In [117]:
M = -1e8

-1.0e8

In [118]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
        end
    end
    for k in 1:nproc
        @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    proM = x * price

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, sum(x .* G) + theta >= g)
        end
    end
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

masterprob (generic function with 1 method)

### Generate cuts

In [119]:
function add_cuts(x, D)
    G = [] # gradient
    g = 0 # intersection
    qy = 0
    
    for d in D
        res = subprob(x, d)
        gradient = -1 * res["pi"]
        
        push!(G, gradient)
        g += d * res["alp"]
        qy += res["qyhat"]
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    z_hat = sum(proD - proS) + qy/size(D)[1]
    Exp_G = sum(G) / size(D)[1]
    Exp_g = sum(g) / size(D)[1]
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>z_lb])
    return cut, z_hat
end


add_cuts (generic function with 1 method)

### L Shape Method

In [120]:
function main(toler)
    i = 0
    z_ub = 1e7
    x_opt = 0
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat, D)
        if z_hat < z_ub
            z_ub = z_hat
            x_opt = x_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    println(z_lb, z_ub, x_opt, i)
    
end

main (generic function with 1 method)

In [121]:
i = 0
z_ub = 1e7
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"];

Academic license - for non-commercial use only - expires 2023-11-27


In [122]:
x_hat, z_lb

([6200.0 0.0 … 0.0 0.0; 1770.291589731998 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 451715.24], -1.0838412704242925e8)

In [102]:
G = [] # gradient
g = 0 # intersection
qy = 0

0

In [87]:
new_cut, z_hat = add_cuts(x_hat, D)
if z_hat < z_ub
    z_ub = z_hat
    x_opt = x_hat
end

push!(cuts, new_cut)

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27


1-element Vector{Any}:
 Dict{String, Any}("gradient" => [-0.12781186121027854 0.08988994647713167 … 2.2113261655335346 3.619925219844523; -0.16449709458296913 -0.0 … 11.774861570087412 8.750448032779437; … ; 2.6613651079346035 1.8976178674364448 … 14.600020319491604 11.441599432116169; -0.0 -0.0 … 5.658420887375434 2.232638351401434], "intersection" => -1.0838412704242925e8)

In [88]:
res = masterprob(cuts)

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 2 entries:
  "x_hat" => [6200.0 0.0 … 0.0 0.0; 1770.29 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.…
  "z_lb"  => -1.08384e8

In [99]:
model = Model(Gurobi.Optimizer)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= -1e8)

for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end
for k in 1:nproc
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 

@objective(model, Min, sum(proD-proS)+theta)
JuMP.optimize!(model)

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 465 rows, 451 columns and 900 nonzeros
Model fingerprint: 0x1d9e757c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 3e+02]
  Bounds range     [1e+08, 1e+08]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 465 rows and 451 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0838413e+08   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.083841270e+08

User-callback calls 33, time in user-callback 0.00 sec


In [95]:
cut = cuts[1]
G = cut["gradient"]
g = cut["intersection"]
# @constraint(model, sum(x .* G) + theta >= g)

-1.0838412704242925e8

In [ ]:
model = Model(Gurobi.Optimizer)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= M)

for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end
for k in 1:nproc
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 

@objective(model, Min, sum(proD-proS)+theta)
JuMP.optimize!(model)

In [104]:
res = subprob(x_hat, D[1])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 0.0
  "pi"    => [0.127812 -0.0898899 … -2.21133 -2.38334; 0.164497 0.0 … -11.7749 …
  "alp"   => 0.0
  "qyhat" => 1.96575e5

In [107]:
res = subprob(x_hat, D[5])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 45171.5
  "pi"    => [0.127812 -0.0898899 … -2.21133 -4.85651; 0.164497 0.0 … -11.7749 …
  "alp"   => 5.0
  "qyhat" => 4.22433e5

In [123]:
G = [] 
g = 0
qy = 0
lsal = []

for d in D
    res = subprob(x_hat, d)
    gradient = -1 * res["pi"]

    push!(G, gradient)
    push!(lsal, res["alp"])
    g += d * res["alp"]
    qy += res["qyhat"]
end

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27


In [124]:
lsal

7-element Vector{Any}:
 0.0
 0.0
 0.0
 6.0
 6.0
 6.0
 6.0

In [125]:
subprob(x_hat, D[1])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 0.0
  "pi"    => [0.127812 -0.0898899 … -2.21133 -2.38334; 0.164497 0.0 … -11.7749 …
  "alp"   => 0.0
  "qyhat" => 1.96575e5

In [126]:
subprob(x_hat, D[2])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 0.0
  "pi"    => [0.127812 -0.0898899 … -2.21133 -2.38334; 0.164497 0.0 … -11.7749 …
  "alp"   => 0.0
  "qyhat" => 1.96575e5

In [127]:
subprob(x_hat, D[3])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 0.0
  "pi"    => [0.127812 -0.0898899 … -2.21133 -2.38334; 0.164497 0.0 … -11.7749 …
  "alp"   => 0.0
  "qyhat" => 1.96575e5

In [128]:
subprob(x_hat, D[4])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 0.0
  "pi"    => [0.127812 -0.0898899 … -2.21133 -5.85651; 0.164497 0.0 … -11.7749 …
  "alp"   => 6.0
  "qyhat" => 1.96575e5

In [132]:
subprob(x_hat, D[5])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 45171.5
  "pi"    => [0.127812 -0.0898899 … -2.21133 -5.85651; 0.164497 0.0 … -11.7749 …
  "alp"   => 6.0
  "qyhat" => 4.67604e5

In [133]:
subprob(x_hat, D[6])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 90343.0
  "pi"    => [0.127812 -0.0898899 … -2.21133 -5.85651; 0.164497 0.0 … -11.7749 …
  "alp"   => 6.0
  "qyhat" => 7.38634e5

In [1]:
T = 451715.24*2

903430.48

In [2]:
subprob(x_hat, T)

LoadError: UndefVarError: subprob not defined

----

### Debug

In [57]:
i = 0
z_ub = 1e7
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]


Academic license - for non-commercial use only - expires 2023-11-27


-1.0838412704242925e8

In [59]:
new_cut, z_hat = get_cuts(x_hat, D)

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27


(Dict{String, Any}("gradient" => [-0.12781186121027854 0.08988994647713167 … 2.2113261655335346 3.619925219844523; -0.16449709458296913 -0.0 … 11.774861570087412 8.750448032779437; … ; 2.6613651079346035 1.8976178674364448 … 14.600020319491604 11.441599432116169; -0.0 -0.0 … 5.658420887375434 2.232638351401434], "intersection" => -1.0838412704242925e8), -7.999337089810846e6)

In [64]:
if z_hat < z_ub
    z_ub = z_hat
    x_opt = x_hat
end

In [65]:
push!(cuts, new_cut)

1-element Vector{Any}:
 Dict{String, Any}("gradient" => [-0.12781186121027854 0.08988994647713167 … 2.2113261655335346 3.619925219844523; -0.16449709458296913 -0.0 … 11.774861570087412 8.750448032779437; … ; 2.6613651079346035 1.8976178674364448 … 14.600020319491604 11.441599432116169; -0.0 -0.0 … 5.658420887375434 2.232638351401434], "intersection" => -1.0838412704242925e8)

In [70]:
model = Model(Gurobi.Optimizer)
set_silent(model)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= M);

Academic license - for non-commercial use only - expires 2023-11-27


In [72]:
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end
for k in 1:nproc
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1);

In [75]:
cuts != []

true

In [76]:
if cuts != []
    for cut in cuts
        G = cut["gradient"]
        g = cut["intersection"]
        println(G)
#         @constraint(model, sum(x .* G) + theta >= g)
    end
end

[-0.12781186121027854 0.08988994647713167 3.3121560494813855 0.08988994647713167 3.0259320489585444 3.112392259152848 3.3121560494813855 2.2101153571782315 2.2101153571782315 3.4991285897879876 2.418769203098346 2.2101153571782315 2.157947673074384 2.2113261655335346 3.619925219844523; -0.16449709458296913 -0.0 2.762316726407633 2.6997787485105107 5.944200275205094 2.2810510915810003 2.762316726407633 11.774158116974029 11.774158116974029 2.5823577517437264 6.213011643978857 11.774158116974029 6.115737229598595 11.774861570087412 8.750448032779437; -0.06253797789712207 -0.0 -0.0 -0.06253797789712207 6.011038783218303 -0.19976379032853783 -0.0 9.011841390566396 9.011841390566396 0.18697254030660201 3.901420521670424 9.011841390566396 3.353420503190962 9.02437790667264 8.296578090373998; -0.1286330854974228 -0.1286330854974228 3.0936330175068316 -0.1286330854974228 6.615186266318056 2.893869227178293 3.0936330175068316 0.9610888723900507 0.9610888723900507 3.2806055578134337 3.4402733703